<center><img src='./figs/cs-logo.png' width=200></center>

<hr style=" border:none; height:3px;">
<h1>
<center>Introduction to Spark </center>
</h1>
<font  size="3" color='#91053d'>
<center >
nacera.seghouani@centralesupelec.fr 
</center>
</font>   
<hr style=" border:none; height:3px;">

# 1. Introduction (PLEASE READ ME)


<p align="justify">
<font size="3">
The purpose of this lab is to acquire  basic **Spark** programming skills that are necessary to develop simple, yet powerful, applications to be executed in a distributed environment.
</font>
</p>


<p align="justify">
<font size="3">
Spark application runs as a set of independent processes (called the _Executors_) across the machines (a.k.a., _Worker_ nodes) of a cluster, on a distributed data, coordinated by the _Driver_, the process that runs the $main()$ function of the application.
</font>
</p>

<p align="justify">
<font size="3">
The _Driver_ creates an object called _SparkContext_ that communicates with the underlying cluster manager and coordinates the distribution of the computation across the _Executors_.
</font>
</p>
<center
<img src="./figs/spark-execution.png" width=400>

<p align="justify">
<font size="3">
To install **Apache Spark **, first Download and Install on your machine (<a href="https://spark.apache.org/downloads.html">see this link</a>).  To use python language, you need **pyspark** package (<a href="https://spark.apache.org/docs/latest/api/python/getting_started/install.html">see this link</a>).
</p>
    <p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**Execute the following cell in order to initialize the _SparkContext_.**</font>
<hr style=" border:none; height:2px;">
</p>

In [2]:
import findspark
findspark.init()
import pyspark
import random
sc = pyspark.SparkContext(appName="lab1")
print("Initialization successful")

23/12/30 11:43:54 WARN Utils: Your hostname, AliGhAbuSaleh resolves to a loopback address: 127.0.1.1; using 192.168.1.33 instead (on interface wlo1)
23/12/30 11:43:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/30 11:43:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Initialization successful


# 2. Resilient Distributed Dataset (RDD)


<p align="justify">
<font size="3">
Spark distributes the data and the computations across the machines of a cluster by using **Resilient Distributed Dataset (RDD)**. 
An RDD is an immutable distributed collection of data. 
Each element of an RDD can be an instance of any Python type, including a user-defined class.
The _SparkContext_  splits an RDD into multiple _partitions_ and scatters them across different machines of the cluster. 
</font>
</p>

<p align="justify">
<font size="3">
The distribution of the partitions of an RDD is completely transparent to the application.
The only thing a Spark application has to do is to create some RDDs and 
specify the computations on these RDDs, by using special functions that Spark provides for this purpose. 
</font>
</p>

## 2.1 Creating RDDs

<p align="justify">
<font size="3">
Spark provides two ways to create an RDD:

<ul>
<li> By distributing a collection of objects.
<li> By loading an external dataset (either in a file or a database).
</ul>
</font>
</p>

<p>
<hr style=" border:none; height:2px;">
 <font size="3" color='#91053d'>**Execute the following code to create an RDD called $words$, where each element is a string taken from the list $wordList$.**</font>
<hr style=" border:none; height:2px;">
</p>

In [3]:
wordList = ["Al", "Ani", "Jackie", "Lalitha", "Mark", "Neil", "Nick", "Shirin"]
words = sc.parallelize(wordList)
print("Done!")

Done!


<p align="justify">
<font size="3">
Once created, Spark provides two types of  operations on a RDD:
<ol>
<li> **Transformations**. A transformation takes in one or more RDDs and returns a new RDD.
<li> **Actions**. An action takes in an RDD and outputs a value.
</ol>
</font>
</p>

<p>
</p>

<p align="justify">
<font size="3">
One common transformation is filtering data that matches a predicate by using the function $filter$.
The function $filter$ is applied on an RDD and takes in a predicate.
It loops through each element of the RDD and verifies whether that element satisfies the predicate. The function $filter$ outputs a new RDD whose elements are those that satisfy the predicate.
</font>
</p>

<p>
<hr style=" border:none; height:2px;">
 <font size="3" color='#91053d'>**Execute the following code to create an RDD called $nNames$ by retaining only the names whose first letter is 'N' from the RDD $words$ .**</font>
<hr style=" border:none; height:2px;">
</p>

In [4]:
'''
As mentioned before, the function filter takes in a predicate that is itself a function (returning a boolean value).

In the code below, the argument of the function filter is a function that takes in a variable called 
"name"; the type of this variable must match the type of the elements of the RDD words, in this case 
a string.
Then the function returns whether the first character of the string "name" is N. 

'''
nNames = words.filter(lambda name: name[:1]=='N') 
print(nNames)
print("done!")

PythonRDD[1] at RDD at PythonRDD.scala:53
done!


<p align="justify">
<font size="3">
The transformation $filter$ above is not executed by Spark until an action is called on the RDD $nNames$.
Spark executes  a transformation on an RDD  when an action is invoked on the RDD itself. This is called _lazy evaluation_. 
    </font>
</p>

<p align="justify">
<font size="3">
One example of action is the function $first()$ that returns the first element in the RDD.
    </font>
</p>

<p>
<hr style=" border:none; height:2px;">
 <font size="3" color='#91053d'>**Execute the following code to print the first element of the RDD $nNames$.**</font>
<hr style=" border:none; height:2px;">
</p>

In [5]:
'''
REMEMBER: the variable nNames has been defined in the cell above, so 
it is VISIBLE in this cell as well as in the cells below the current one
'''
print(nNames.first())


Neil


<p align="justify">
<font size="3">
The previous example  shows that a Spark application is  a sequence of operations that create, transform and perform  actions on RDDs.
</font>
</p>

<p align="justify">
<font size="3">
The following code shows an example of creation of RDD from an external dataset, a Neo4j database log text file. 
The function $textFile()$ takes in the path to the input text file and returns an RDD, where each element is a line of the file.
The code goes through the following steps:

<ol>
<li> **RDD creation**. Creates an RDD called $lines$, where each element is a line from the input text file.
<li> **RDD filter**. Creates an RDD called $exceptions$ from the RDD $lines$ by only retaining the lines containing the string "exception".
<li> **Action count()**. Counts the number of elements of the RDD $exceptions$. 
<li> **Print first line**. Prints the first line of the RDD $exceptions$. 
</ol>
</font>
</p>

<p>
</p>

<p align="justify">
<font size="3">
Here we also see  the _lazy evaluation_ of transformations. 
Spark does not execute the function $textFile()$ as soon as it is invoked.
Instead, it waits until the first action $count()$ is invoked.
</font>
</p>

<p>
<hr style=" border:none; height:2px;">
 <font size="3" color='#91053d'>**Execute the following code.**</font>
 <hr style=" border:none; height:2px;">
</p>

In [6]:
data_file = "./data/neo4j.log"

# 1. RDD creation
lines = sc.textFile(data_file)

#2. RDD filter
exceptions = lines.filter(lambda line : "exception" in line)

#3. Action count()
nbLines = exceptions.count()
print("Number of exception lines ", nbLines)

#4. Print first line.
exceptions.take(1)


Number of exception lines  5


['2017-10-06 13:12:26.097+0000 ERROR Failed to start Neo4j: Starting Neo4j failed: Component \'org.neo4j.server.database.LifecycleManagingDatabase@2814f71a\' was successfully initialized, but failed to start. Please see the attached cause exception "Format version is not supported (resource BufferedChecksumIndexInput(MMapIndexInput(path="/Users/quercini_gia/Documents/software/neo4j-community-3.2.5/data/databases/social-network/upgrade/index/lucene/relationship/crosslinks/segments_1"))): -11 (needs to be between 1071082519 and 1071082519). This version of Lucene only supports indexes created with release 4.0 and later.". Starting Neo4j failed: Component \'org.neo4j.server.database.LifecycleManagingDatabase@2814f71a\' was successfully initialized, but failed to start. Please see the attached cause exception "Format version is not supported (resource BufferedChecksumIndexInput(MMapIndexInput(path="/Users/quercini_gia/Documents/software/neo4j-community-3.2.5/data/databases/social-network/u

## 2.2 Transformations

<p align="justify">
<font size="3">
Here are some common transformations in Spark. 
In the following list, $r$ denotes the RDD on which the transformation is invoked.
<ol>
<li> $r.filter()$. Returns an RDD consisting of only the elements of the input RDD $r$ that satisfy a predicate.
<li> $r.map(func)$. Applies a function $func$ to each element of the input RDD $r$ and returns an RDD of the result.
<li> $r.flatMap(func)$. Same as $map()$, but used when $map()$ would return an RDD where each element is a list.
<li> $r.union(other)$. Takes in two RDDs ($r$ and $other$) and returns an RDD that contains the elements from both. Unlike the mathematical operation, $union$ in Spark does not remove the duplicates.
<li> $r.intersection(other)$. Takes in two RDDs ($r$ and $other$) and returns an RDD that contains the elements found in both.
<li> $r.subtract(other)$. Takes in two RDDs ($r$ and $other$) and returns an RDD that contains the elements from the RDD $r$, except those that are found in $other$.
<li> $r.cartesian(other)$. Takes in two RDDs ($r$ and $other$) and returns an RDD that contains the Cartesian product of both.
<li> $r.distinct()$. Returns an RDD that contains the same elements as the input RDD $r$ without duplicates.
</ol>
    
We are now going to look at an example of use of these transformations.
</font>    
</p>


<p>
<hr style=" border:none; height:2px;">
 <font size="3" color='#91053d'>**Execute the following code to create the two RDDs $r1$ and $r2$**</font>
<hr style=" border:none; height:2px;">
</p>

In [7]:
r1 = sc.parallelize([1, 2, 3, 4])
r2 = sc.parallelize([3, 4, 5, 6, 7])
print("done")

done


### Use of $map()$

<p align="justify">
<font size="3">
Here we see an example of use of the transformation $map()$.
</font>
</p>

<p>
<hr style=" border:none; height:2px;">
<font size="3" color='#91053d'>**Execute the following code to create:
 <ol>
 <li> an RDD $square$, where each element is the square of the corresponding element in $r1$; 
 <li> an RDD $half$, where each element is the half of the corresponding element in $r2$.**
 </ol>
 </font>
 <hr style=" border:none; height:2px;">
</p>

In [8]:
square = r1.map(lambda x : x*x)
half = r2.map(lambda x: x/2)

# The function collect() is an action that transforms the RDD into a Python list that can be printed.
print("Elements of RDD square ", square.collect())
print("Elements of the RDD half ", half.collect())

Elements of RDD square  [1, 4, 9, 16]
Elements of the RDD half  [1.5, 2.0, 2.5, 3.0, 3.5]


### Use of $flatMap()$

<p align="justify">
<font size="3">
The transformation $flatMap()$ works by applying the transformation $map()$ on the input RDD; 
if each element of the resulting RDD is a list, $flatMap()$ returns an RDD where all lists are merged.
In other words, when $map()$ returns an RDD where the elements are lists, $flatMap()$ returns an RDD where the elements are the values of the list.
</font>
</p>

<p align="justify">
<font size="3">
Let's see an example. Suppose that we want to return an RDD from $r1$ where each element is associated to its square. 
More precisely, the output RDD will be as follows:
<center>
[ [1, 1], [2, 4], [3, 9], [4, 16] ]
</center>
</font>    
</p>

<p>
<hr style=" border:none; height:2px;">
 <font size="3" color='#91053d'>**Execute the following code to create an RDD $squares$ where each element of $r1$ is associated to its square.**</font>
<hr style=" border:none; height:2px;">
</p>

In [9]:
squares = r1.map(lambda x : [x, x*x])
print("Elements of RDD squares ", squares.collect())

Elements of RDD squares  [[1, 1], [2, 4], [3, 9], [4, 16]]


<p align="justify">
<font size="3">

As you can see, each element of the RDD $squares$ is a list of two elements; indeed, 
after calling the action $collect()$, we obtain a list of lists in Python. 
In order to obtain a simple list of elements, we flatten the lists with $flatMap()$.
More precisely, with $flatMap()$ we obtain the following RDD:
<center>
[1, 1, 2, 4, 3, 9, 4, 16]
</center>
</font>
</p>

<p>
<hr style=" border:none; height:2px;">

 <font size="3" color='#91053d'>**Execute the following code to create an RDD $squares$ where each element of $r1$ is associated to its square (but each element is just a value instead of a list of values)**</font>
<hr style=" border:none; height:2px;">

In [10]:
squares = r1.flatMap(lambda x : [x, x*x])
print("Elements of RDD squares ", squares.collect())

Elements of RDD squares  [1, 1, 2, 4, 3, 9, 4, 16]


### Use of set transformations

<p align="justify">
<font size="3">
The set transformations are $union$, $intersection$, $subtract$ and $cartesian$.
</font>
</p>

<p>
<hr style=" border:none; height:2px;">
 <font size="3" color='#91053d'>**Execute the following code to see an example of these transformations**</font>
<hr style=" border:none; height:2px;">

In [11]:
union = r1.union(r2)
intersection = r1.intersection(r2)
subtract = r1.subtract(r2)
cartesian = r1.cartesian(r2)

print("Elements of RDD r1 ", r1.collect())
print("Elements of RDD r2 ", r2.collect())
print("Elements of RDD union ", union.collect())
print("Elements of RDD intesection ", intersection.collect())
print("Elements of RDD subtract ", subtract.collect())
print("Elements of RDD cartesian ", cartesian.collect())


Elements of RDD r1  [1, 2, 3, 4]
Elements of RDD r2  [3, 4, 5, 6, 7]
Elements of RDD union  [1, 2, 3, 4, 3, 4, 5, 6, 7]


Elements of RDD intesection  [3, 4]


Elements of RDD subtract  [1, 2]
Elements of RDD cartesian  [(1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7)]


### Use of $distinct()$

<p align="justify">
<font size="3">
The transformation $distinct()$ returns an RDD that contains the same elements as the input RDD without the duplicates.
</font>
</p>

<p>
<hr style=" border:none; height:2px;">

 <font size="3" color='#91053d'>**Execute the following code to see an example of use of $distinct()$**</font>
 <hr style=" border:none; height:2px;">
</p>

In [12]:
nodup = union.distinct()
print("Elements of the RDD union: ", union.collect())
print("Elements of the RDD union (with no duplicates): ", nodup.collect())

Elements of the RDD union:  [1, 2, 3, 4, 3, 4, 5, 6, 7]
Elements of the RDD union (with no duplicates):  [1, 2, 3, 4, 5, 6, 7]


## 2.3. Actions

<p align="justify">
<font size="3">
Here are some common actions in Spark. 
As with transformations, $r$ denotes the RDD on which the action is invoked.

<ol>
<li> $r.reduce(func)$. Performs a pair-wise application of the given function $func$ to the elements of the input RDD $r$.
<li> $r.collect()$. Returns a list with all the elements of the input RDD $r$.
<li> $r.count()$. Returns the number of elements in the input RDD $r$.
<li> $r.countByValue()$. Returns the number of times each element occurs in the input RDD $r$.
<li> $r.take(num)$. Prints the first $num$ elements of the input RDD $r$.
<li> $r.top(num)$. Prints the top $num$ elements of the input RDD $r$ (sorted in decreasing order).
</ol>
</font>
</p>

### Use of $reduce(func)$

<p align="justify">
<font size="3">
The action $reduce(func)$ performs a pair-wise application of the given function $func$ on the elements of the input RDD. 
</font>
</p>

<p>
<hr style=" border:none; height:2px;">
<font size="3" color='#91053d'>**Execute the following code to sum all values of the RDD $r1$**</font>
<hr style=" border:none; height:2px;">


In [13]:
'''
The function passed to the reduce MUST take in two arguments 
that have the same type as the elements of the input RDD
'''
sum = r1.reduce(lambda x, y : x + y)
print("Elements of the RDD r1 ", r1.collect())
print("Sum of the elements of the RDD r1: ", sum)


Elements of the RDD r1  [1, 2, 3, 4]
Sum of the elements of the RDD r1:  10


### Use of $countByValue()$

<p align="justify">
<font size="3">
The action $countByValue()$ counts the number of the occurrences of each element of the input RDD.
The result is a Python dictionary, where a key is an element of the input RDD and the corresponding value the number of its occurrences.
</font>
</p>

<p>
<hr style=" border:none; height:2px;">

<font size="3" color='#91053d'>**Execute the following code to get the number of occurrences of each element in the RDD $union$**</font>

<hr style=" border:none; height:2px;">
</p>

In [14]:
occurrences = union.countByValue()
print("Elements of the RDD union ", union.collect())
print("Occurrences of each element in the RDD union:")
for k, v in occurrences.items():
    print(k, " --> ",  v, " occurrences")

Elements of the RDD union  [1, 2, 3, 4, 3, 4, 5, 6, 7]
Occurrences of each element in the RDD union:
1  -->  1  occurrences
2  -->  1  occurrences
3  -->  2  occurrences
4  -->  2  occurrences
5  -->  1  occurrences
6  -->  1  occurrences
7  -->  1  occurrences


## 3. Exercises 

<p align="justify">
<font size="3">
Now it's your turn! 
You're going to apply yourself the transformations and the actions that we've seen so far. 
To this extent, we will use the file **./data/moby-dick.txt** that contains the text of the famous novel by Herman Melville. 
The goal of this exercise is to count the number of words in the file and the number of occurrences of each word.
</font>
</p>
<p align="justify">
<font size="3">
First, we are going to preprocess the input text, by removing punctuation and special characters and to lowercase all words.
</font>
</p>    

<hr style=" border:none; height:2px;">

###  Exercise 1

<p align="justify">
<font size="3">
The function $preprocess()$ defined below takes in an RDD that contains the lines of the input text file and returns an RDD where each element is a word. In the code below, the first step of the function $preprocess()$, which removes the non-letter characters from the input RDD by using a regular expression, is already implemented.
</font>
</p>

<p align="justify">
<font size="3" color='#91053d'>
**Complete the function $proprocess()$ with the following steps:**
<ol start="2">
<li>    Apply a transformation on the RDD $text$ to obtain an RDD $words$, where each element is a word from the file.
<li>    Filter out from the RDD $words$ the words with length 0.
<li>    Lowercase all words of the RDD $words$.
<li>    Return the RDD $words$.
</ol>
Execute the code, which will print the number of words in the input text file and the top-100 most frequent words.
</font>
</p>

<hr style=" border:none; height:2px;">

In [15]:
import operator;
import re;


def preprocess(text):
    '''
    Regular expression for removing all non-letter characters in the file.
    '''
    regex = re.compile('[^a-zA-Z ]')
    '''
    Step 1. 
    Remove the non-letter characters.
    After this transformation, mobydick is an RDD where each element is a line of the file
    '''
    text = text.map(lambda line: regex.sub('', line))
    
    '''
    **************************************************
    COMPLETE STEP 2 THROUGH 4
    **************************************************
    '''
    # Apply a transformation on the RDD $text$ to obtain an RDD $words$, where each element is a word from the file.
    words = text.flatMap(lambda word: word.split(" "))
    # Filter out from the RDD $words$ the words with length 0.
    words_no_zero = words.filter(lambda word: len(word) > 0)
    # Lowercase all words of the RDD $words$.
    words = words_no_zero.map(lambda word: word.lower()) 
    
    '''
    Step 5.
    Returns the RDD words.
    '''
    return words


# Reads the novel into a RDD
mobydick = sc.textFile('./data/moby-dick.txt')

words = preprocess(mobydick)

print("Number of words ", words.count())

'''
Top-100 most frequent words
'''
# countByValue() counts the number of occurrences of each word.
# It returns a dictionary. 
# The function sorted sorts by value and returns a list of tuples (w, f), where w is a word 
# and f its number of occurrences.
# The list will be sorted in ascending order of number of occurrences
occurrences = sorted(words.countByValue().items(), key=operator.itemgetter(1))

# Reverse the order
occurrences.reverse()
print("Top-100 most frequent words")
i = 0
for (w, f) in occurrences:
    i += 1
    if i > 100:
        break
    print(w,"--->",f)
    

Number of words  208240
Top-100 most frequent words
the ---> 14065
of ---> 6440
and ---> 6257
a ---> 4535
to ---> 4491
in ---> 4050
that ---> 2900
his ---> 2485
it ---> 2343
i ---> 1930
but ---> 1763
he ---> 1729
as ---> 1698
with ---> 1690
is ---> 1672
was ---> 1618
for ---> 1574
all ---> 1464
this ---> 1351
at ---> 1295
by ---> 1169
not ---> 1121
from ---> 1071
him ---> 1035
so ---> 1030
on ---> 1028
be ---> 1024
whale ---> 891
one ---> 875
you ---> 851
had ---> 765
now ---> 759
there ---> 755
have ---> 751
or ---> 686
were ---> 674
they ---> 639
which ---> 617
then ---> 614
me ---> 606
some ---> 604
their ---> 603
when ---> 591
are ---> 584
my ---> 584
an ---> 583
like ---> 567
no ---> 566
upon ---> 560
what ---> 537
into ---> 517
out ---> 509
more ---> 499
up ---> 496
if ---> 469
its ---> 457
them ---> 450
old ---> 436
man ---> 433
we ---> 426
would ---> 421
ahab ---> 417
ye ---> 416
been ---> 408
over ---> 397
other ---> 393
these ---> 387
ship ---> 378
will ---> 377
only ---> 369

<hr style=" border:none; height:2px;">

### Exercise 2

<p align="justify">
<font size="3">
In the previous example, you've probably noticed that the top-100 most frequent words are _functions_ words, such as articles, pronouns, conjunctions. These words have little lexical meaning and express a grammatical relationship with the other words. 
In many applications, these functions words are not useful and they can be ignored.
For instance, when you submit a query against a search engine (e.g., "restaurants in Paris"), function words (e.g., "in") are not useful to retrieve the Web pages relevant to the query.
</font>
</p>

<p align="justify">
<font size="3">
In text processing applications, function words are part of a list of _stop words_ that include all words that are frequently used and give little semantic contribution to the content of a text.
Since there isn't any agreement as to the definition of _stop word_, it's possible to find many such lists on the Web.
</font>
</p>

<p align="justify">
<font size="3" color='#91053d'>**Write a function $preprocess()$ that applies the same operations as the function that you defined above and, in addition, removes the stop words. This function $preprocess()$ takes in: (i) an RDD that contains the lines of the input text file (as above) and (ii) an RDD containing the stop words. The function returns an RDD with the words of the input text file. **</font>
</p>

<p align="justify">
<font size="3">
HINT: The file with the list of stop words is **./data/stopwords.txt**. All stop words are already lowercased.
</font>
</p>

<hr style=" border:none; height:2px;">

In [16]:
def preprocess(text, stopwords):
    '''
    Regular expression for removing all non-letter characters in the file.
    '''
    regex = re.compile('[^a-zA-Z ]')
    '''
    Step 1. 
    Remove the non-letter characters.
    After this transformation, mobydick is an RDD where each element is a line of the file
    '''
    text = text.map(lambda line: regex.sub('', line))
    
    '''
    Step 2.
    Obtain the RDD containing the words in the file.
    Note that the function line.split() returns the list of words in a line. 
    Therefore, each element of the RDD words would be a list, shouldn't we use flatMap.
    '''
    words = text.flatMap(lambda line: line.split(" "))

    '''
    Step 3.
    Filter out the words with length 0.
    '''
    words = words.filter(lambda word: len(word) > 0)
    
    '''
    Step 4.
    Lowercase all words of the RDD $words$
    '''
    words = words.map(lambda word: word.lower())
    
    '''
    Step 5 Remove stop words.
    '''
    '''
    **************************************************
    COMPLETE STEP 5
    **************************************************
    '''
    words = words.subtract(stopwords)
    
    # Returns the words
    return words


# Load the stopwords to an RDD
stopwords = sc.textFile("./data/stopwords.txt")
words = preprocess(mobydick, stopwords)
print("Number of words after stopword removal ", words.count())

# Count the number of occurrences as before.
occurrences = sorted(words.countByValue().items(), key=operator.itemgetter(1))
# Reverse the order
occurrences.reverse()
print("Top-100 most frequent words")
i = 0
for (w, f) in occurrences:
    i += 1
    if i > 100:
        break
    print(w, ", ", f)
    


Number of words after stopword removal  102358
Top-100 most frequent words
whale ,  891
one ,  875
old ,  436
man ,  433
ahab ,  417
ye ,  416
ship ,  378
whales ,  363
sea ,  363
though ,  362
time ,  321
long ,  313
captain ,  292
great ,  289
said ,  288
seemed ,  280
must ,  279
two ,  278
last ,  273
head ,  263
see ,  257
way ,  253
white ,  247
little ,  247
boat ,  241
round ,  239
three ,  235
sperm ,  232
first ,  229
stubb ,  227
men ,  224
every ,  223
say ,  223
well ,  221
us ,  221
much ,  218
queequeg ,  211
good ,  195
hand ,  194
side ,  183
go ,  183
thing ,  181
look ,  178
boats ,  175
made ,  174
away ,  173
chapter ,  172
come ,  170
starbuck ,  169
many ,  161
deck ,  158
water ,  158
far ,  157
day ,  156
seen ,  156
eyes ,  153
ships ,  152
sir ,  151
sort ,  151
cried ,  149
back ,  148
world ,  147
part ,  147
thought ,  147
know ,  145
oh ,  137
whole ,  137
right ,  136
aye ,  135
life ,  134
air ,  133
crew ,  133
thus ,  132
soon ,  128
night ,  128
thee

# 4. Pair RDDs 

<p align="justify">
<font size="3">
Pair RDDs are simply RDDs where each element is a key-value pair. They are useful for expressing _MapReduce_ computations. 
</font>
</p>

<p align="justify">
<font size="3">
We are now going to look at examples of  transformations and actions on pair RDDs.
</font>
</p>

## 4.1 Creation of Pair RDDs

<p align="justify">
<font size="3">
One common way to create a pair RDD is to transform an existing RDD with a $map()$. 
</font>
</p>

<hr style=" border:none; height:2px;">
<p align="justify">
<font size="3" color='#91053d'>**Execute the following code to create a Pair RDD $kvwords$ from the RDD $words$. Each element of $kvwords$ is a pair where the key is a word and the value is 1**</font>
</p>
<hr style=" border:none; height:2px;">

In [17]:
kvwords = words.map(lambda word : (word, 1))
print(kvwords.take(100))

[('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('years', 1), ('yea

## 4.2 Transformations on Pair RDDs


<p align="justify">
<font size="3">
All transformations applied to standard RDDs can be applied to Pair RDDs as well. The only difference is that any function that is passed to a transformation must take in tuples instead of single values.
</font>
</p>

<p align="justify">
<font size="3">
In addition, the following transformations can **only** be applied to Pair RDDs:
<ol>
<li> $r.reduceByKey(func)$. It applies the given function $func$ pairwise to all elements of the input RDD $r$ that are associated to the same key. 
<li> $r.sortBy(func, asc)$. Returns an RDD where the elements of the input RDD $r$ are sorted according to the given criteria.
<li> $r.groupByKey()$. Groups values with the same key from the input RDD $r$.
<li> $r.keys()$. Returns an RDD where the elements are the keys from the input RDD $r$.
<li> $r.values()$. Returns an RDD where the elements are the values from the input RDD $r$.
</ol>
</font>
</p>

<hr style=" border:none; height:2px;">
<font size="3" color='#91053d'>**Execute the following code to count the number of occurrences of each word in the RDD $kvwords$ and print the top-100 most frequent words**</font>
<hr style=" border:none; height:2px;">

In [18]:
'''
kvwords is an RDD where each element is (w, 1), w is a word from the novel Moby Dick.
If a word w occurs k times, there will be k pairs (w, 1) in the RDD kvwords.
To compute the number of occurrences of each word, 
we should just sum the 1s associated to all pairs with the same key, for all keys.
This is achieved with the reduceByKey() function.
'''
occurrences = kvwords.reduceByKey(lambda x, y : x + y)

'''
Each element of the RDD occurrences is a pair (w, f), where w is a word and f is the number of occurrences of w.
Finally, we sort the RDD occurrences by value by using sortBy(). 
The first argument of the function sortBy() is a function that takes in a key-value pair and returns the 
value (x[1]). 
The second argument specifies the order (ascending or descending).
'''
occurrences = occurrences.sortBy(lambda x: x[1], ascending=False)
occurrences.take(100)

[('whale', 891),
 ('one', 875),
 ('old', 436),
 ('man', 433),
 ('ahab', 417),
 ('ye', 416),
 ('ship', 378),
 ('sea', 363),
 ('whales', 363),
 ('though', 362),
 ('time', 321),
 ('long', 313),
 ('captain', 292),
 ('great', 289),
 ('said', 288),
 ('seemed', 280),
 ('must', 279),
 ('two', 278),
 ('last', 273),
 ('head', 263),
 ('see', 257),
 ('way', 253),
 ('little', 247),
 ('white', 247),
 ('boat', 241),
 ('round', 239),
 ('three', 235),
 ('sperm', 232),
 ('first', 229),
 ('stubb', 227),
 ('men', 224),
 ('say', 223),
 ('every', 223),
 ('us', 221),
 ('well', 221),
 ('much', 218),
 ('queequeg', 211),
 ('good', 195),
 ('hand', 194),
 ('go', 183),
 ('side', 183),
 ('thing', 181),
 ('look', 178),
 ('boats', 175),
 ('made', 174),
 ('away', 173),
 ('chapter', 172),
 ('come', 170),
 ('starbuck', 169),
 ('many', 161),
 ('water', 158),
 ('deck', 158),
 ('far', 157),
 ('seen', 156),
 ('day', 156),
 ('eyes', 153),
 ('ships', 152),
 ('sort', 151),
 ('sir', 151),
 ('cried', 149),
 ('back', 148),
 ('tho


<p align="justify">
<font size="3">
Another way to obtain the same result, would be to:
<ol>
<li> Apply the transformation $groupByKey()$ to the RDD $kvwords$. The result is a RDD $occurrences$, where each element is a pair $(w, L)$, $w$ is a word and $L$ is a list of 1s (as many as the number of occurrences of $w$).
<li> Apply a transformation $map()$ to the RDD $occurrences$ to obtain a new RDD $occurrences$, where each element is a pair $(w, len(L))$, $w$ being a word and $len(L)$ being the number of elements of $L$.
<li> Sort the RDD $occurrences$ as before.
</ol>
</font>
</p>

<hr style=" border:none; height:2px;">
<font size="3" color='#91053d'>**Execute the following code to see the result with this alternative solution**</font>
<hr style=" border:none; height:2px;">

In [19]:
occurrences = kvwords.groupByKey()
occurrences = occurrences.map(lambda x : (x[0], len(x[1])))
occurrences = occurrences.sortBy(lambda x: x[1], ascending=False)
occurrences.take(100)

[('whale', 891),
 ('one', 875),
 ('old', 436),
 ('man', 433),
 ('ahab', 417),
 ('ye', 416),
 ('ship', 378),
 ('sea', 363),
 ('whales', 363),
 ('though', 362),
 ('time', 321),
 ('long', 313),
 ('captain', 292),
 ('great', 289),
 ('said', 288),
 ('seemed', 280),
 ('must', 279),
 ('two', 278),
 ('last', 273),
 ('head', 263),
 ('see', 257),
 ('way', 253),
 ('little', 247),
 ('white', 247),
 ('boat', 241),
 ('round', 239),
 ('three', 235),
 ('sperm', 232),
 ('first', 229),
 ('stubb', 227),
 ('men', 224),
 ('say', 223),
 ('every', 223),
 ('us', 221),
 ('well', 221),
 ('much', 218),
 ('queequeg', 211),
 ('good', 195),
 ('hand', 194),
 ('go', 183),
 ('side', 183),
 ('thing', 181),
 ('look', 178),
 ('boats', 175),
 ('made', 174),
 ('away', 173),
 ('chapter', 172),
 ('come', 170),
 ('starbuck', 169),
 ('many', 161),
 ('water', 158),
 ('deck', 158),
 ('far', 157),
 ('seen', 156),
 ('day', 156),
 ('eyes', 153),
 ('ships', 152),
 ('sort', 151),
 ('sir', 151),
 ('cried', 149),
 ('back', 148),
 ('tho

## 4.3 Actions on Pair RDDs

<p justify="align">
<font size="3">
As with the transformations, all actions available for standard RDDs can be used on Pair RDDs as well.
In addition, the following actions can be performed on Pair RDDs:
<ol>
<li> $countByKey()$. Counts the number of values associated with the same key. Returns a dictionary.
<li> $collecAsMap()$. Collects the RDD as a dictionary (in the same way as the function $collect()$ returns a list from a standard RDD).
<li> $lookup(key)$. Returns a list with all the values associated with the given _key_.
</ol>
</font>
</p>

<p justify="align">
<font size="3">
Referring again to the example of counting the occurrences of each word, the following code is yet another way to solve the problem by using the action $countByKey()$ on the RDD $kvwords$.
</font>
</p>

<hr style=" border:none; height:2px;">
<font size="3" color='#91053d'>**Execute the following code**</font>
<hr style=" border:none; height:2px;">

In [20]:
occurrences = sorted(kvwords.countByKey().items(), key=operator.itemgetter(1))
# Reverse the order
occurrences.reverse()
print("Top-100 most frequent words")
i = 0
for (w, f) in occurrences:
    i += 1
    if i > 100:
        break
    print(w, ", ", f)
    

Top-100 most frequent words
whale ,  891
one ,  875
old ,  436
man ,  433
ahab ,  417
ye ,  416
ship ,  378
whales ,  363
sea ,  363
though ,  362
time ,  321
long ,  313
captain ,  292
great ,  289
said ,  288
seemed ,  280
must ,  279
two ,  278
last ,  273
head ,  263
see ,  257
way ,  253
white ,  247
little ,  247
boat ,  241
round ,  239
three ,  235
sperm ,  232
first ,  229
stubb ,  227
men ,  224
every ,  223
say ,  223
well ,  221
us ,  221
much ,  218
queequeg ,  211
good ,  195
hand ,  194
side ,  183
go ,  183
thing ,  181
look ,  178
boats ,  175
made ,  174
away ,  173
chapter ,  172
come ,  170
starbuck ,  169
many ,  161
deck ,  158
water ,  158
far ,  157
day ,  156
seen ,  156
eyes ,  153
ships ,  152
sir ,  151
sort ,  151
cried ,  149
back ,  148
world ,  147
part ,  147
thought ,  147
know ,  145
oh ,  137
whole ,  137
right ,  136
aye ,  135
life ,  134
air ,  133
crew ,  133
thus ,  132
soon ,  128
night ,  128
thee ,  128
tell ,  128
take ,  126
came ,  126
thi

# 5. Exercises

<p align="justify">
<font size="3">
Now it's your turn to apply the notions that you've learned so far to create some more complex applications.
</font>
</p>

<hr style=" border:none; height:2px;">

##  Exercise 3

<p align="justify">
<font size="3">
In the folder _./data/bbc_ you'll find a collection of 50 documents from the BBC news website corresponding to stories in five topics from 2004-2005. The five topics are: _business_, _entertainment_, _politics_, _sport_ and _tech_. 
In the directory, the stories are text files (named _001.txt_, _002.txt_...) organized into five directories, one for topic.
</font>
</p>

<p align="justify">
<font size="3">
In this exercise, we want to create an **inverted index**, one that associates each word to the list of files the word occurs in.
More precisely, for each word, the inverted index will have a list of the names  of the files (path relative to the folder _./data_) that contain the word. The figure below shows the entry in the index for the word "family".

<img src="./figs/inverted-index.png" width=400>
    An inverted index is an essential component of a search engine. In fact, given any word, the inverted index allows the search engine to quickly retrieve all documents containing that word.
</font>
</p>

<p align="justify">
<font size="3">
The idea is to use a **MapReduce** schema. 
The **Map** task will create a key-value pair $(w, d)$ for each word $w$ that appears in the document $d$. $d$ is the path of the document relative to the folder _./data_ (e.g., _./data/bbc/business/001.txt_).
The **Reduce** task will group by key all the key-value pairs with the same key to form the inverted index.
</font>
</p>

<p align="justify">
<font size="3" color='#91053d'>**Write a Spark application to create an inverted index for the BBC collection. By using the function $lookup$ (already provided), print the documents that contain the word 'family'.**
<br>
HINT:
<ol>
<li> Your program needs to go through all the documents in each of the five folders of the BBC collection. Click <a href="https://stackoverflow.com/questions/10377998/how-can-i-iterate-over-files-in-a-given-directory">here</a> to find out how to loop over the files of a directory in Python.
<li> To preprocess the text of a document, you can use the function $preprocess()$ defined above. The RDD $words$ returned by this function contains a word as many times as it appears in a document. Since for an inverted index we just need to know that a word is in a specific document, you should remove the duplicates from the RDD $words$.
</ol>
</font>
</p>

<hr style=" border:none; height:2px;">

In [30]:
os.environ["PYTHONHASHSEED"]=str(232)

In [31]:

import os
'''
**************************************************
    COMPLETE THE PROGRAM
**************************************************
'''
# make an inverted index 
# loop over all the documents in the directory ./data/bbc and subfolders and use the function preprocess to obtain the RDD words.
# Then, compute the number of words in each document and print the result.
# define the inversted index
inverted_index = None
for root, dirs, files in os.walk("./data/bbc"):
    for file in files:
        if file.endswith(".txt"):
            path = os.path.join(root, file)
            # print(path)
            # load the file
            text = sc.textFile(path)
            # apply preprocess
            words = preprocess(text, stopwords)
            # drop duplicates words in the path
            words = words.distinct()
            # create the inverted index for the words in the path relative to the file
            # if the inverted index is empty, create it
            if inverted_index is None:
                inverted_index = words.map(lambda word : (word, [path]))
            # otherwise, update it
            else:
                inverted_index = inverted_index.union(words.map(lambda word : (word, [path])))
# reduce the inverted index to have word -> [path1, path2, ...]
inverted_index = inverted_index.reduceByKey(lambda x, y : x + y)

# print the inverted index key-value pairs
print("Inverted index", inverted_index.collect())
        



Inverted index [('everything', ['./data/bbc/sport/009.txt', './data/bbc/tech/009.txt']), ('backed', ['./data/bbc/tech/010.txt', './data/bbc/tech/006.txt']), ('policy', ['./data/bbc/tech/010.txt', './data/bbc/politics/005.txt', './data/bbc/business/002.txt']), ('step', ['./data/bbc/tech/001.txt']), ('orange', ['./data/bbc/tech/005.txt', './data/bbc/entertainment/007.txt']), ('led', ['./data/bbc/tech/006.txt']), ('matters', ['./data/bbc/politics/007.txt']), ('mpc', ['./data/bbc/politics/005.txt']), ('globalisation', ['./data/bbc/business/008.txt']), ('ears', ['./data/bbc/business/002.txt']), ('plates', ['./data/bbc/entertainment/001.txt']), ('barnett', ['./data/bbc/entertainment/006.txt']), ('m', ['./data/bbc/sport/009.txt', './data/bbc/sport/003.txt', './data/bbc/sport/010.txt', './data/bbc/sport/001.txt', './data/bbc/sport/007.txt', './data/bbc/sport/005.txt', './data/bbc/sport/006.txt', './data/bbc/politics/006.txt', './data/bbc/business/003.txt', './data/bbc/business/001.txt', './dat

In [38]:
# print the inverted index keys 
print("Inverted index keys", inverted_index.keys().collect())

Inverted index keys ['everything', 'backed', 'policy', 'step', 'orange', 'led', 'matters', 'mpc', 'globalisation', 'ears', 'plates', 'barnett', 'm', 'van', 'weekend', 'jadel', 'prove', 'faces', 'longcourse', 'demystify', 'media', 'surprisngly', 'marketing', 'lists', 'gone', 'today', 'allows', 'branch', 'involvement', 'oldfashioned', 'retraining', 'crops', 'quarterly', 'unnatural', 'guise', 'raise', 'soccer', 'ian', 'third', 'grand', 'karl', 'created', 'pushed', 'although', 'bulbs', 'blairs', 'altria', 'regulators', 'greenback', 'broughton', 'upward', 'playwright', 'version', 'since', 'also', 'energy', 'twice', 'santry', 'officially', 'axe', 'comptia', 'attacker', 'baftawinning', 'concept', 'juggle', 'accusations', 'funny', 'cochairman', 'sandra', 'chose', 'channel', 'margins', 'definition', 'sexuality', 'robbie', 'got', 'turn', 'selected', 'mccambridge', 'kluft', 'laptop', 'size', 'tell', 'proponents', 'referendum', 'polling', 'effective', 'changed', 'prevents', 'details', 'thefirst', 

In [43]:
# look up for word family
word = 'family'
paths = inverted_index.lookup(word)
print("Word ", word, " appears in ", paths)

Word  family  appears in  []


<hr style=" border:none; height:2px;">

##  Exercise 4

<p align="justify">
<font size="3">
Given the BBC collection, we want to calculate the **co-occurrence matrix** $M$, such that $M[w_1][w_2]$ is the number of documents in which two words $w_1$ and $w_2$ appear together.
</font>
</p>

<p align="justify">
<font size="3">
A co-occurrence matrix is generally sparse (i.e., lot of entries are zero), because we expect the pairs of words that have no co-occurrence to be far more than the ones that have at least one co-occurrence. It is certainly wise to represent only the non-zero elements of the matrix.
To this extent, we use a **MapReduce** schema.
</font>
</p>

<p align="justify">
<font size="3">
For any two words $w_i$, $w_j$ that co-occur in the same document, the **Map** task will output a key-value pair $((w_i, w_j), 1)$, where the key is the pair of co-occurring words $(w_i, w_j)$ and the value is 1.
Note that the two key-value pairs $((w_i, w_j), 1)$ and $((w_j, w_i), 1)$ ought to be considered as having the same key, because the order of occurrence of the two words does not matter (the only thing that matters is whether the two words co-occur in the same document). 
For this reason, the key-value pair $((w_i, w_j), 1)$ returned by the Map task is such that $w_i < w_j$, where $<$ is the lexicographic order.
</font>
</p>

<p align="justify">
<font size="3">
In the output of the Map task, a key-value pair $((w_i, w_j), 1)$ appears as many times as documents where $w_i$ and $w_j$ co-occur. In order to obtain the number of co-occurrences of $w_i$ and $w_j$, the **Reduce** task will just have to sum the values associated to the key $(w_i, w_j)$.
</font>
</p>



<p align="justify">
<font size="3" color='#91053d'>**Write a Spark application that creates the co-occurrence matrix of the documents in the BBC collection. Print the top-20 most frequent co-occurrences.**
<br>
HINT: as before, use the function $preprocess()$ and remove the duplicate words.
</font>
</p>

<hr style=" border:none; height:2px;">

In [44]:
'''
**************************************************
    COMPLETE THE PROGRAM
**************************************************
'''
# loop over all the documents in the directory ./data/bbc and subfolders and use the function preprocess to obtain the RDD words.
# Then, create co-occurrence pairs for all pairs of words in the same document.
# Finally, compute the number of co-occurrences for each pair of words.
# define the co-occurrence pairs
co_occurrence_pairs = None
for root, dirs, files in os.walk("./data/bbc"):
    for file in files:
        if file.endswith(".txt"):
            path = os.path.join(root, file)
            # print(path)
            # load the file
            text = sc.textFile(path)
            # apply preprocess
            words = preprocess(text, stopwords)
            ''' 
            Do  I need to drop duplicates words in the path?
            '''
            # # drop duplicates words in the path
            # words = words.distinct()
            # create the co-occurrence pairs for the words in the path relative to the file
            # if the co-occurrence pairs is empty, create it
            if co_occurrence_pairs is None:
                co_occurrence_pairs = words.cartesian(words)
            # otherwise, update it
            else:
                co_occurrence_pairs = co_occurrence_pairs.union(words.cartesian(words))
# reduce the co-occurrence pairs to have word1-word2 -> count
co_occurrence_pairs = co_occurrence_pairs.map(lambda pair : (pair, 1)).reduceByKey(lambda x, y : x + y)

# print the top 20 co-occurrence pairs
print("Top-20 co-occurrence pairs", co_occurrence_pairs.takeOrdered(20, key=lambda x: -x[1]))



Top-20 co-occurrence pairs [(('said', 'said'), 42), (('also', 'also'), 30), (('also', 'said'), 24), (('said', 'also'), 24), (('world', 'world'), 23), (('new', 'new'), 22), (('one', 'one'), 21), (('world', 'said'), 20), (('said', 'world'), 20), (('said', 'new'), 19), (('year', 'year'), 19), (('new', 'said'), 19), (('set', 'set'), 18), (('last', 'last'), 18), (('year', 'said'), 17), (('said', 'year'), 17), (('world', 'also'), 17), (('years', 'years'), 17), (('time', 'time'), 17), (('also', 'world'), 17)]


<hr style=" border:none; height:2px;">

##  Exercise 5

<p align="justify">
<font size="3">
One important task in the analysis of textual data is the computation of the similarity between two textual documents.
Intuitively, the more words two textual documents share, the more similar they are.
</font>
</p>

<p align="justify">
<font size="3">
Let $d_1$ and $d_2$ be two documents, and let $W(d_1)$ and $W(d_2)$ be the set of words in $d_1$ and $d_2$, respectively. The similarity $S(d_1, d_2)$ between $d_1$ and $d_2$ can be computed with the _Jaccard_ score as follows:
</font>
</p>

<p>
</p>

$$S(d_1, d_2) = \frac{W(d_1) \cap W(d_2)}{W(d_1) \cup W(d_2)}$$


<p align="justify">
<font size="3" color='#91053d'>**Write a function $jaccard$ that takes in two RDDs containing the words of two documents and returns the Jaccard similarity score of the documents. Test the function on the documents of the BBC.**
</font>
</p>

<hr style=" border:none; height:2px;">

In [45]:
'''
**************************************************
    COMPLETE THE PROGRAM
**************************************************
'''
# define a function jaccard that takes two rdds contatin the workds of two documents and computes the Jaccard similarity between the two documents
def jaccard(rdd1, rdd2):
    # compute the number of words in the first document
    n1 = rdd1.count()
    # compute the number of words in the second document
    n2 = rdd2.count()
    # compute the number of words in the intersection of the two documents
    n12 = rdd1.intersection(rdd2).count()
    # compute the Jaccard similarity
    return n12 / (n1 + n2 - n12)

In [46]:

# test the function of the documents in the directory ./data/bbc and subfolders
for root, dirs, files in os.walk("./data/bbc"):
    for file in files:
        if file.endswith(".txt"):
            path = os.path.join(root, file)
            # print(path)
            # load the file
            text = sc.textFile(path)
            # apply preprocess
            words = preprocess(text, stopwords)
            # compute the Jaccard similarity between the document and the document ./data/bbc/business/001.txt
            print("Jaccard similarity between ", path, " and ./data/bbc/business/001.txt is ", jaccard(words, preprocess(sc.textFile("./data/bbc/business/001.txt"), stopwords)))

Jaccard similarity between  ./data/bbc/sport/008.txt  and ./data/bbc/business/001.txt is  0.014534883720930232
Jaccard similarity between  ./data/bbc/sport/009.txt  and ./data/bbc/business/001.txt is  0.026030368763557483
Jaccard similarity between  ./data/bbc/sport/003.txt  and ./data/bbc/business/001.txt is  0.03309692671394799
Jaccard similarity between  ./data/bbc/sport/010.txt  and ./data/bbc/business/001.txt is  0.02077922077922078
Jaccard similarity between  ./data/bbc/sport/001.txt  and ./data/bbc/business/001.txt is  0.022284122562674095
Jaccard similarity between  ./data/bbc/sport/007.txt  and ./data/bbc/business/001.txt is  0.009345794392523364
Jaccard similarity between  ./data/bbc/sport/002.txt  and ./data/bbc/business/001.txt is  0.015432098765432098
Jaccard similarity between  ./data/bbc/sport/005.txt  and ./data/bbc/business/001.txt is  0.01483679525222552
Jaccard similarity between  ./data/bbc/sport/004.txt  and ./data/bbc/business/001.txt is  0.02654867256637168
Jacca

<hr style=" border:none; height:2px;">

##  Exercise 6

<p align="justify">
<font size="3">
We are now going to use the similarity function that we defined in Exercise 4 to create a simple text classifier.
A _text classifier_ is an application that takes in a textual document and determines the topic that the document covers.
Usually, text classification is based on supervised machine learning algorithms, like Naive Bayes and SVM; 
since these algorithms are out of the scope of this course, we limit ourselves to a simple classifier that assigns a document to the topic with the highest similarity to the document itself.
</font>
</p>

<p align="justify">
<font size="3">
More precisely, let's consider five documents $d_1, \ldots, d_5$, one for each topic of the BBC dataset; arbitrarily, we say that each of these documents is representative of one topic. 
Now, consider a document $d$ that is different from the representative documents. 
The text classifier computes the similarity of $d$ to $d_1, \ldots, d_5$, selects the document $d_i$ that has the highest similarity to $d$ and assigns $d$ to the topic covered by $d_i$.
</font>
</p>

<p align="justify">
<font size="3" color='#91053d'>**Write a function $classify$ that takes in a document $d$ (already preprocessed) and the list of representative documents (already preprocessed) for each topic and outputs the predicted topic of $d$.**
</font>
</p>
<p>
<font size="3" color='#91053d'>
HINT: You can use the files named _001.txt_ as the representative documents for each topic.   
</font>
</p>

<hr style=" border:none; height:2px;">

In [50]:
'''
**************************************************
    COMPLETE THE PROGRAM
**************************************************
'''
# define a function classify, that take document and list of representtive documents and returns the class of the document
def classify(document, classes):
    # compute the Jaccard similarity between the document and each representative document
    similarities = []
    for representative in classes:
        similarities.append(jaccard(document, representative))
    # return the class of the document
    return similarities.index(max(similarities))

# define the list of representative documents ( take 001.txt from each subfolder of ./data/bbc)
representatives = []
for root, dirs, files in os.walk("./data/bbc"):
    for file in files:
        if file.endswith("001.txt"):
            path = os.path.join(root, file)
            representatives.append(preprocess(sc.textFile(path), stopwords))
            # print the path of the representative document and it's class index
            print(path, " --> ", len(representatives) - 1)
            

# test the function of the documents in the directory ./data/bbc and subfolders ( random document from each subfolder)
for root, dirs, files in os.walk("./data/bbc"):
    for file in files:
        if file.endswith(".txt"):
            path = os.path.join(root, file)
            # print(path)
            # load the file
            text = sc.textFile(path)
            # apply preprocess
            words = preprocess(text, stopwords)
            # compute the class of the document
            print("Class of ", path, " is ", classify(words, representatives))
            # skip to the next subfolder
            break
        


./data/bbc/sport/001.txt  -->  0
./data/bbc/tech/001.txt  -->  1
./data/bbc/politics/001.txt  -->  2
./data/bbc/business/001.txt  -->  3
./data/bbc/entertainment/001.txt  -->  4
Class of  ./data/bbc/sport/008.txt  is  0
Class of  ./data/bbc/tech/008.txt  is  2
Class of  ./data/bbc/politics/008.txt  is  2
Class of  ./data/bbc/business/008.txt  is  0
Class of  ./data/bbc/entertainment/008.txt  is  2


<hr style=" border:none; height:2px;">

##  Exercise 7 

<p align="justify">
<font size="3">

After implementing a classifier, we need to evaluate it. The evaluation is based on the **confusion matrix** that describes the performance of the classifier on a test dataset for which the true values are known.
An example of confusion matrix is shown in the figure below. 
Each row contains the instances in a predicted class while each column represents the instances in an actual class.
From the matrix in the Figure, we learn that there are 10 documents that are in the class "business" (see the total of the first column), of which 5 are correctly predicted as being in the class "business", 2 are incorrectly classified in the class "entertainment" and  3 are incorrectly classified in the class "sport".
</font>
<img src="./figs/confusion-matrix.png" width=800>
</p>

<p align="justify">
<font size="3">
For each class $C$, we can define the notion of true positives (TP), false positives (FP) and false negatives (FN) for that class (the figure shows the TP, FP and FN for the class "Business"):
<ul>
<li> **True positives** (TP). Set of documents whose actual and predicted class is $C$.
<li> **False positives** (FP). Set of documents whose predicted class is $C$, while the actual class is not $C$.
<li> **False negatives** (FN). Set of documents whose predicted class is not $C$, while the actual class is $C$.
</ul>
</font>
</p>

<p align="justify">
<font size="3">
</font>
</p>

<p align="justify">
<font size="3">
For each class $C$, we can compute three evaluation measures, called _precision_ ($P_C$), _recall_ ($R_C$) and 
_f-measure_ ($F_C$), as follows:
</font>
</p>

$$P_C = \frac{|TP|}{|TP| + |FP|}\quad R_C = \frac{|TP|}{|TP| + |FN|}\quad F_C = 2\cdot \frac{P_C\cdot R_C}{P_C + R_C}$$

<p align="justify">
<font size="3">
The _precision_ $P_C$ tells how many of the documents classified in $C$ are correctly classified;
the _recall_ $R_C$ tells how many of the documents that are actually in $C$ are correctly classified in $C$;
the _f-measure_ is an harmonic mean of _precision_ and _recall_ and gives an indication of the overall _accuracy_ of the classifier.
</font>
</p>


<p align="justify">
<font size="3" color='#91053d'>**Write a program that evaluates the classifier implemented in the previous exercise on the whole BBC dataset. Print precision, recall and f-measure for each class.
Compare different classifiers using different representative documents for the topics**
</font>
</p>


<hr style=" border:none; height:2px;">

In [65]:
# define the function to evaluate the classification, print precision, recall and f1-score for each class
def evaluate(classes, representatives):
    # define the confusion matrix
    confusion_matrix = [[0 for i in range(len(representatives))] for j in range(len(representatives))]
    # loop over all the documents in the directory ./data/bbc and subfolders
    for root, dirs, files in os.walk("./data/bbc"):
        for file in files:
            if file.endswith(".txt"):
                path = os.path.join(root, file)
                # print(path)
                # load the file
                text = sc.textFile(path)
                # apply preprocess
                words = preprocess(text, stopwords)
                # compute the class of the document
                predicted = classify(words, representatives)
                # compute the real class of the document
                real = classes.index(root.split("/")[-1])
                # update the confusion matrix
                confusion_matrix[real][predicted] += 1
    # return the confusion matrix
    return confusion_matrix

In [66]:
# use the function evaluate to evaluate the classification
classes = ["business", "entertainment", "politics", "sport", "tech"]
confusion_matrix = evaluate(classes, representatives)


In [67]:
# print the confusion matrix
print("Confusion matrix")
for row in confusion_matrix:
    print(row)

# compute precision, recall and f1-score for each class
for i in range(len(representatives)):
    # compute the precision
    precision = confusion_matrix[i][i] / __builtins__.sum(confusion_matrix[i])
    # compute the recall
    recall = confusion_matrix[i][i] / __builtins__.sum([row[i] for row in confusion_matrix])
    # compute the f1-score
    if precision + recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * precision * recall / (precision + recall)
    # print the results
    print("Class ", classes[i], " precision ", precision, " recall ", recall, " f1-score ", f1_score)

Confusion matrix
[2, 0, 0, 8, 0]
[4, 1, 3, 1, 1]
[2, 0, 8, 0, 0]
[10, 0, 0, 0, 0]
[4, 1, 5, 0, 0]
Class  business  precision  0.2  recall  0.09090909090909091  f1-score  0.12500000000000003
Class  entertainment  precision  0.1  recall  0.5  f1-score  0.16666666666666669
Class  politics  precision  0.8  recall  0.5  f1-score  0.6153846153846154
Class  sport  precision  0.0  recall  0.0  f1-score  0
Class  tech  precision  0.0  recall  0.0  f1-score  0
